In [ ]:
#download the QA dataset
#!kaggle competitions download -c chaii-hindi-and-tamil-question-answering
#!unzip train.csv.zip

In [1]:
import pandas as pd
import warnings
import transformers

from pathlib import Path
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator

warnings.filterwarnings('ignore')

In [2]:
BASE = Path("..")
df = pd.read_csv(BASE / 'train.csv')
df_test = pd.read_csv(BASE / 'test.csv')

In [3]:
df.sample(10)

,id,context,question,answer_text,answer_start,language
9,7a6e807d7,"நெல்சன் மண்டேலா (Nelson Rolihlahla Mandela, 18...",நெல்சன் மண்டேலா எத்தனை ஆண்டுகள் சிறையில் இருந்...,27,494,tamil
946,e30235cab,आईबीएम (English: International Business Machin...,आईबीएम कंपनी का मुख्यालय कहाँ पर है?,"संयुक्त राष्ट्र के अर्मोंक, न्यू यॉर्क में",176,hindi
862,2a75aabcd,Main Page\n\nआई एन एस (INS) विक्रमादित्य (San...,आई एन एस विक्रमादित्य को किस वर्ष भारतीय नौसेन...,16 नवम्बर 2013,279,hindi
252,67b7f2604,பூனை பாலூட்டி இனத்தைச் சேர்ந்த ஒரு ஊனுண்ணி ஆகு...,பூனையின் சராசரி ஆயுட்காலம் என்ன?,12-15,3784,tamil
969,9aa6b3e1e,होली (Holi) वसंत ऋतु में मनाया जाने वाला एक मह...,हिंदू धर्म के रंगों के त्यौहार का नाम क्या है?,होली,0,hindi
462,4eb7216bf,बसरा इराक का तीसरा सबसे बड़ा नगर एवं महत्वपूर्...,बसरा नगर किस देश में स्थित है?,इराक,5,hindi
1062,063be4c5a,जयद्रथ-वध मैथिलीशरण गुप्त द्वारा रचित प्रसिद्ध...,'जयद्रथ-वध'' ग्रंथ किसके द्वारा रचित है?,मैथिलीशरण गुप्त,10,hindi
1035,4dc78516e,एमा चारलॉट डुएरे वॉटसन (जन्म 15 अप्रैल 1990) ए...,एम्मा वाटसन की राष्ट्रीयता क्या है?,ब्रिटिश,48,hindi
460,f88f5c088,क्रिकेट एक बल्ले और गेंद का दलीय खेल है जिसकी ...,क्रिकेट के खेल में कितने खिलाड़ी होते है?,ग्यारह,2697,hindi
614,3ba572984,टेनिस खेल 2 टीमों के बीच गेंद से खेले जाने वाल...,टेनिस की शुरुआत कहाँ हुई?,फ्रांस,373,hindi


In [5]:
dataset = Dataset.from_pandas(df[['id','context', 'question', 'answer_text', 'answer_start']],split='train')

In [9]:
dataset = dataset.train_test_split(test_size=.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answer_text', 'answer_start'],
        num_rows: 891
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answer_text', 'answer_start'],
        num_rows: 223
    })
})

In [10]:
dataset.shape

{'train': (891, 5), 'test': (223, 5)}

In [12]:
dataset['train'][385]

{'id': 'c18b77850',
 'context': 'जापान, एशिया महाद्वीप में स्थित देश है। जापान चार बड़े और अनेक छोटे द्वीपों का एक समूह है। ये द्वीप एशिया के पूर्व समुद्रतट, यानि प्रशांत महासागर में स्थित हैं। इसके निकटतम पड़ोसी चीन, कोरिया तथा रूस हैं। जापान में वहाँ का मूल निवासियों की जनसंख्या ९८.५% है। बाकी 0.5% कोरियाई, 0.4 % चाइनीज़ तथा 0.6% अन्य लोग है। जापानी अपने देश को निप्पॉन कहते हैं, जिसका मतलब सूर्योदय है। जापान की राजधानी टोक्यो है और उसके अन्य बड़े महानगर योकोहामा, ओसाका और क्योटो हैं। बौद्ध धर्म देश का प्रमुख धर्म है और जापान की जनसंख्या में 96% बौद्ध अनुयायी है।[1][2]\n इतिहास \n\nजापानी लोककथाओं के अनुसार विश्व के निर्माता ने सूर्य देवी तथा चन्द्र देवी को भी रचा। फिर उसका पोता क्यूशू द्वीप पर आया और बाद में उनकी संतान होंशू द्वीप पर फैल गए।\n प्राचीन काल \nजापान का प्रथम लिखित साक्ष्य ५७ ईस्वी के एक चीनी लेख से मिलता है। इसमें एक ऐसे राजनीतिज्ञ के चीन दौरे का वर्णन है, जो पूर्व के किसी द्वीप से आया था। धीरे-धीरे दोनों देशों के बीच राजनैतिक और सांस्कृतिक सम्बंध स्थापित हुए। उस समय जा

In [15]:
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
batch_size = 16

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['sop_classifier.classifier.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [17]:
tokenizer("This is my pc?", "I'll be using Indic BERT")

{'input_ids': [2, 255, 54, 1283, 8, 90956, 293, 3, 7922, 398, 13348, 121, 8, 8017, 26, 326, 3814, 8, 121763, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
max_length = 512
doc_stride = 128 

In [38]:
for i, example in enumerate(dataset['train']):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = dataset['train'][17]

In [39]:
len(tokenizer(example["question"], example["context"])["input_ids"])

9229

In [40]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

512

In [41]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [42]:
[len(x) for x in tokenized_example["input_ids"]]

[512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 512,
 445]

In [43]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] कतन अमरक उपनवश अमरक करत क हसस थ?[SEP] अमरक करनतकर यदध (1775–1783), जस सयकत रजय म अमरक सवतनतरत यदध य करनतकर यदध भ कह जत ह, गरट बरटन और उसक तरह उततर अमरक उपनवश क बच एक सनय सघरष थ, जसस व उपनवश सवतनतर सयकत रजय अमरक बन। शरआत लडई उततर अमरक महदवप पर हई। सपतवरषय यदध म परजय क बद, बदल क लए आतर फरनस न 1778 म इस नए रषटर स एक सनध क, ज अतत वजय क लए नरणयक सबत हई। अमरक क सवततरत यदध न यरपय उपनवशवद क इतहस म एक नय मड ल दय। उसन अफरक, एशय एव लटन अमरक क रजय क भव सवततरत क लए एक पदधत तयर कर द। इस परकर अमरक क यदध क परणम कवल इतन ह नह हआ क 13 उपनवश मतदश बरटन स अलग ह गए बलक व उपनवश एक तरह स नए रजनतक वचर तथ ससथओ क परयगशल बन गए। पहल बर 16व 17व शतबद क यरपय उपनवशवद और वणजयवद क चनत दकर वजय परपत क। अमरक उपनवश क इगलड क आधपतय स मकत क लए सघरष, इतहस क अनय सघरष स भनन थ। यह सघरष न त गरब स उतपनन असतष क परणम थ और न यह क जनत समतवद वयवसथ स पडत थ। अमरक उपनवश न अपन सवचछदत और वयवहर म सवततरत बनए रखन क लए इगलड सरकर क कठर औपनवशक नत क वरदध सघरष कय थ। अमरक क सवततरत सगरम वशव इतहस क एक महतवपरण घटन ह। करत स परव अमरक क सथत इगलड एव सपन

In [44]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 1), (2, 4), (5, 10), (11, 12), (13, 16), (16, 17), (18, 21), (21, 26), (27, 28), (29, 36), (37, 39), (40, 44), (45, 46), (47, 49), (50, 51), (0, 0), (0, 4), (5, 6), (7, 12), (13, 18), (19, 21), (22, 23), (24, 25), (25, 27), (27, 29), (29, 31), (31, 32), (32, 35), (35, 37), (37, 38), (38, 42), (43, 47), (48, 51), (51, 55), (56, 57), (57, 59), (61, 66), (67, 68), (69, 71), (72, 75), (76, 79), (79, 80), (81, 83), (84, 85), (86, 87), (87, 89), (90, 95), (96, 101), (102, 104), (105, 106), (107, 108), (108, 110), (111, 114), (115, 119), (120, 122), (123, 124), (124, 128), (129, 130), (130, 134), (135, 138), (138, 141), (141, 144), (144, 145), (146, 151), (151, 154), (155, 157), (157, 162), (163, 164), (165, 168), (168, 169), (170, 173), (175, 177), (178, 182), (182, 185), (185, 189), (190, 191), (191, 193), (194, 196), (197, 198), (198, 200), (201, 202), (202, 206), (206, 207), (208, 210), (211, 214), (214, 215), (216, 219), (219, 221), (222, 225), (226, 229), (229, 233), (234, 

In [45]:
first_token_id = tokenized_example["input_ids"][0][77]
offsets = tokenized_example["offset_mapping"][0][77]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

▁स 


In [46]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [47]:
answers = example["answer_text"]
start_char = example["answer_start"]
end_char = start_char + len(example["answer_text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

64 64


In [50]:
pad_on_right = tokenizer.padding_side == "right"
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answer_text"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(examples["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = examples["answer_start"][0]
            end_char = start_char + len(examples["answer_text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [51]:
features = prepare_train_features(dataset['train'][:5])

In [53]:
tokenized_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=dataset['train'].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [54]:
model = AutoModelForQuestionAnswering.from_pretrained('ai4bharat/indic-bert')

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForQuestionAnswering: ['sop_classifier.classifier.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly

In [55]:
model_name = "ai4bharat/indic-bert".split("/")[-1]
args = TrainingArguments(
    f"chaii-base",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)
data_collator = default_data_collator

In [59]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()